# Weather Information Agent

The Weather Information Agent is designed to interpret natural language weather-related queries, extract the location, fetch real-time weather data, and generate a user-friendly response.

- Location Extraction: Uses OpenAI API to determine the location from user queries.

- Weather Data Retrieval: Fetches real-time weather details such as temperature, humidity, wind speed, and cloud coverage using Weatherstack API.

- Natural Language Response: Generates conversational weather descriptions using OpenAI’s language model.

- Comprehensive Query Processing: Ensures an end-to-end flow from understanding the user’s question to delivering an insightful weather report.

This agent provides an interactive and seamless weather update experience for users.

### 1. Install necessary packages

In [ ]:
!pip install openai python-dotenv

  Attempting uninstall: openai
    Found existing installation: openai 1.3.0
    Uninstalling openai-1.3.0:
      Successfully uninstalled openai-1.3.0


### 2. Import Dependencies

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

### 3. Load the environment variables

In [ ]:
load_dotenv()

# set up API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

### 4. Weather Agent Class
The WeatherAgent class is a Python implementation that provides weather information for a given location using natural language processing. It combines the OpenAI API for language understanding and the Weatherstack API for retrieving weather data.

### Key Components
#### Initialization
- The class requires API keys for both Weatherstack and OpenAI services
- These keys are used to authenticate API requests
#### Methods 

**1. call_openai_api**
- Makes requests to OpenAI's API
- Sends messages in a specific format required by OpenAI
- Returns the generated text response or None if an error occurs
- Handles API errors and exceptions gracefully 

**2. extract_location**
- Uses OpenAI to extract location information from a natural language query
- Provides specific instructions to the AI to focus only on location extraction
- Returns the extracted location name 

**3. get_weather_data**
- Fetches current weather data from Weatherstack API for a specified location
- Processes the API response to extract relevant weather information
- Returns a structured dictionary with key weather metrics including:
  - Location name and country
  - Temperature and "feels like" temperature
  - Humidity and wind speed
  - Weather description and cloud cover

  **4. generate_weather_response**
- Creates a natural language description of weather conditions
- Uses OpenAI to convert structured weather data into conversational text
- Returns a friendly, informative weather description 

**5. process_weather_query**
- Orchestrates the entire weather information process
- Extracts location from user query
- Retrieves weather data for that location
- Generates a natural language response
- Handles errors at each step with appropriate fallback messages

In [ ]:
class WeatherAgent:
    def __init__(self, weather_api_key, openai_api_key):
        """
        Initialize the Weather Agent with API keys.
        
        Args:
            weather_api_key (str): API key for weather service
            openai_api_key (str): API key for OpenAI
        """
        self.weather_api_key = weather_api_key
        self.openai_api_key = openai_api_key
    
    def call_openai_api(self, messages, model="gpt-3.5-turbo"):
        """
        Call OpenAI API directly using requests.
        
        Args:
            messages (list): List of message objects
            model (str): Model to use
            
        Returns:
            str: Response content
        """
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.openai_api_key}"
        }
        
        payload = {
            "model": model,
            "messages": messages,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                data=json.dumps(payload)
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            else:
                print(f"Error calling OpenAI API: {response.status_code}")
                print(response.text)
                return None
        except Exception as e:
            print(f"Exception when calling OpenAI API: {e}")
            return None

    def extract_location(self, user_query):
        """
        Extract location from user query using OpenAI.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Extracted location name
        """
        messages = [
            {"role": "system", "content": "You are a location extraction assistant. Extract the specific city or location from the given query."},
            {"role": "user", "content": f"Extract the city or location name from this query: '{user_query}'. Only return the location name, nothing else."}
        ]
        
        try:
            return self.call_openai_api(messages)
        except Exception as e:
            print(f"Error extracting location: {e}")
            return None

    def get_weather_data(self, location):
        """
        Fetch weather data for a given location using Weatherstack API.
        
        Args:
            location (str): City name or location identifier
        
        Returns:
            dict: Processed weather information
        """
        base_url = "http://api.weatherstack.com/current"
        params = {
            'access_key': self.weather_api_key,
            'query': location,
            'units': 'm'  # Metric units
        }
        
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            
            # Check for errors in the response
            if 'error' in data:
                print(f"Weather API error: {data['error']['info']}")
                return None
            
            # Extract relevant weather information
            weather_info = {
                'location': data['location']['name'],
                'country': data['location']['country'],
                'temperature': data['current']['temperature'],
                'feels_like': data['current']['feelslike'],
                'humidity': data['current']['humidity'],
                'description': data['current']['weather_descriptions'][0] if data['current']['weather_descriptions'] else 'No description available',
                'wind_speed': data['current']['wind_speed'],
                'cloudiness': data['current']['cloudcover']
            }
            
            return weather_info
        
        except Exception as e:
            print(f"Error fetching weather data: {e}")
            if 'response' in locals() and hasattr(response, 'text'):
                print(f"Response content: {response.text}")
            return None

    def generate_weather_response(self, weather_data):
        """
        Generate a natural language response for weather data.
        
        Args:
            weather_data (dict): Weather information
        
        Returns:
            str: Descriptive weather response
        """
        messages = [
            {"role": "system", "content": "You are a friendly weather narrator. Create a conversational weather description."},
            {"role": "user", "content": f"Create a friendly, informative weather description using this data: {weather_data}"}
        ]
        
        try:
            return self.call_openai_api(messages)
        except Exception as e:
            print(f"Error generating weather response: {e}")
            return None

    def process_weather_query(self, user_query):
        """
        Process a complete weather query from extraction to response.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Comprehensive weather information response
        """
        # Extract location
        location = self.extract_location(user_query)
        if not location:
            return "Sorry, I couldn't determine the location from your query."
        
        # Get weather data
        weather_data = self.get_weather_data(location)
        if not weather_data:
            return f"Sorry, I couldn't retrieve weather data for {location}."
        
        # Generate natural language response
        weather_response = self.generate_weather_response(weather_data)
        return weather_response or "I encountered an issue generating the weather description."

### 5. Initialize the Agent

In [ ]:
weather_agent = WeatherAgent(
    weather_api_key=WEATHER_API_KEY, 
    openai_api_key=OPENAI_API_KEY
)

In [22]:
# Process a query
query = "What's the weather like in New York today?"
print(f"Query: {query}")
result = weather_agent.process_weather_query(query)
print(result)

Query: What's the weather like in New York today?
Hey there! Today in New York, United States of America, we've got a cozy 6 degrees Celsius with a feels-like temperature of 4 degrees. It's a bit on the cloudy side with overcast skies and a humidity level of 49%. The wind is blowing gently at 6 km/h, and the cloud cover is at 100%. It's a perfect day to grab a warm drink and enjoy the calm weather. Stay cozy and have a wonderful day!


In [23]:
query = "Tell me about the current conditions in London"
print(f"Query: {query}")
result = weather_agent.process_weather_query(query)
print(result)

Query: Tell me about the current conditions in London
Hey there, folks in London, United Kingdom! Today, it's a lovely day with a temperature of 17°C, which feels just right at 17°C. The skies are looking partly cloudy, adding a touch of charm to the day. The humidity is at a comfortable 48%, so it's not too muggy out there.

You might feel a gentle breeze with a wind speed of 4 km/h, giving a pleasant touch to the atmosphere. The cloudiness is at 50%, so you can expect some sunny breaks throughout the day. Overall, it's a great day to enjoy a walk in the park or a cup of tea outdoors. Stay cozy and enjoy the delightful weather in London!
